<a href="https://colab.research.google.com/github/ucfilho/ANN_raianars/blob/master/ANN_parte_016_M%C3%A9tricas_classifica_OUT_07_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# este comando clona o diretorio github ucfilho/ANN no colaboratory research
!git clone https://github.com/RaianaSeixas/ANN #clonar do Github
%cd ANN
#!ls
print("Hello world"+"\n"+"I'm evolving!!!"+"\n")

%matplotlib inline

fatal: destination path 'ANN' already exists and is not an empty directory.
/content/ANN
Hello world
I'm evolving!!!



In [2]:
!ls

000_ANN_Gera_Matriz_out_04_2019.ipynb		    X_DAT_B_set_17_2019.csv
000_ANN_parte_001_XY_dados_out_01_2019.ipynb	    X_DAT_set_17_2019.csv
000_ANN_parte_002_XY_Métricas_D10_OUT_4_2019.ipynb  XY_D10_30_out_7_2019.csv
000_ANN_parte_009_Métricas_PCA_OUT_07_2019.ipynb    XY_D10_out_1_2019.csv
000_Gerenciador_DE_var_F_CR_STAT_set_27_2019.ipynb  XY_D10_out_5_2019.csv
imput_ANN_D10_30_out_07_19.zip			    Y_DAT_B_set_17_2019.csv
imput_ANN_D10_6F.zip				    Y_DAT_set_17_2019.csv
imput_ANN_D10_out_05_19.zip


In [0]:
# Functions
def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

def tasig(x,deriv=False):
    if(deriv==True):
        return 1-x*x
    return 2/(1+np.exp(-2*x))-1

# purelin function
def purelin(x,deriv=False):
    if(deriv==True):
        return 1
    #N=len(x)
    #for i in range(N):
      #if (x[i]>1):
        #x[i]=1
    return x

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [0]:
def ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,X_train,y):
  syn0 = 2*np.random.random((Neuron_input,Neuron_hidden)) - 1
  syn1 = 2*np.random.random((Neuron_hidden,Neuron_output)) - 1
  for j in range(Epoch):
    l0 = X_train
    l1 = tasig(np.dot(l0,syn0))
    l2 = purelin(np.dot(l1,syn1))
    l2_error = y - l2
    l2_delta = l2_error*purelin(l2,deriv=True)
    l1_error = l2_delta.dot(syn1.T)
    l1_delta = l1_error * tasig(l1,deriv=True)
    syn1 += l1.T.dot(l2_delta)*Learning_rate
    syn0 += l0.T.dot(l1_delta)*Learning_rate
  return syn0,syn1



In [0]:
def ANN_ycal(syn0,syn1,X_train):
  l0 = X_train
  l1 = tasig(np.dot(l0,syn0))
  l2 = purelin(np.dot(l1,syn1))
  y_calc=np.reshape(l2,len(l2))
  return y_calc

In [0]:
def Classifica(ycalc):
  n=len(ycalc)
  for i in range(n):
    if(abs(ycalc[i])<0.05):
      ycalc[i]=0
    elif (ycalc[i]<0):
      ycalc[i]=-1
    else:
      ycalc[i]=1
  return ycalc

In [0]:
# leitura dos dados
DAT=pd.read_csv("XY_D10_30_out_7_2019.csv")
#print(DAT.head())

In [0]:
# separa as colunas de interesse
Index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)
#print(Dat_Select)
#print(Dat_Select.head())

In [0]:
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [0]:
ColF=DAT_OLD['F']
W=np.array(DAT_OLD['F'])
Wo=np.array(DAT_OLD['Fo'])
for i in range(len(W)):
  W[i]=W[i]-Wo[i]
  if(abs(W[i])<0.05):
    W[i]=0
  elif (W[i]<0):
    W[i]=-1
  else:
    W[i]=1
DAT['F'] = W

In [13]:
# obtem as variaveis independentes (treino e teste)
# separa os dados em treino e teste
train=DAT.sample(frac=0.8,random_state=None) 
test=DAT.drop(train.index)
IND=[1,2,3,4,6,7,8,9,10,11,12,13,14]
x_train=train.iloc[:,IND]
x_test=test.iloc[:,IND]
X_OLD=DAT_OLD.iloc[:,IND]
X_NOME=DAT_OLD.columns[IND]
print(X_NOME)


Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')


In [0]:
# obtem as variaveis dependentes (treino e teste)
Index=[15] #F
#Index=[16] #CR
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]
#print(y_train.head())

In [0]:
y_train=np.array(y_train)
Nomes=x_train.columns
x_test=np.array(x_test)
x_train=np.array(x_train)

In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.001
omega=0.9
Epoch=20000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_calc_test=ANN_ycal(syn0,syn1,x_test)

In [17]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)
print("====================================")
y_calc_train = Classifica(y_calc_train)
y_calc_test = Classifica(y_calc_test)
print("====================================")
print("Teste " )
print(confusion_matrix(y_calc_test,y_obs_test))
print("Treino " )
print(confusion_matrix(y_calc_train,y_obs_train))


Epocas= 20000 Learning rate= 0.001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')
Teste 
[[4 0 7]
 [0 0 0]
 [2 5 7]]
Treino 
[[48  5  0]
 [ 0  8  0]
 [ 1  2 37]]


In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.0001
omega=0.9
Epoch=100000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_obs_test=y_test
y_calc_test=ANN_ycal(syn0,syn1,x_test)



In [19]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)
print("====================================")
y_calc_train=Classifica(y_calc_train)
y_calc_test=Classifica(y_calc_test)
print("====================================")
print("Teste " )
print(confusion_matrix(y_calc_test,y_obs_test))
print("Treino ")
print(confusion_matrix(y_calc_train,y_obs_train))

Epocas= 100000 Learning rate= 0.0001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')
Teste 
[[6 3 7]
 [0 1 0]
 [0 1 7]]
Treino 
[[47  5  0]
 [ 2  4  0]
 [ 0  6 37]]


In [20]:
print(X_NOME)

Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')


In [21]:
# ========= Novas mudancas ==================================================
IND=[1,2,6,7]
x_train=train.iloc[:,IND]
x_test=test.iloc[:,IND]
X_OLD=DAT_OLD.iloc[:,IND]
X_NOME=DAT_OLD.columns[IND]
print(X_NOME)
y_train=np.array(y_train)
x_test=np.array(x_test)
x_train=np.array(x_train)

Index(['DI RELATIVO', 'FRAC Its', 'Fo', 'CRo'], dtype='object')


In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.0001
omega=0.9
Epoch=300000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_obs_test=y_test
y_calc_test=ANN_ycal(syn0,syn1,x_test)



In [23]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)
print("====================================")
y_calc_train=Classifica(y_calc_train)
y_calc_test=Classifica(y_calc_test)
print("====================================")
print("Teste " )
print(confusion_matrix(y_calc_test,y_obs_test))
print("Treino " )
print(confusion_matrix(y_calc_train,y_obs_train))



Epocas= 300000 Learning rate= 0.0001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'Fo', 'CRo'], dtype='object')
Teste 
[[ 6  2  1]
 [ 0  0  2]
 [ 0  3 11]]
Treino 
[[43  7  0]
 [ 2  2  0]
 [ 4  6 37]]
